## In this notebook I will try to implement the loss of the Negative Contrastive estimation. 

Notice that eventually we will also have to change the model to add the parameters of the normalising constant.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os, sys


sys.path.insert(1, "./../util")
sys.path.insert(1, "./../model")
from encoded_protein_dataset_new import get_embedding, EncodedProteinDataset_new, collate_fn_new#, dynamic_collate_fn
from pseudolikelihood import get_npll2, get_npll_indep
import torch, torchvision
from torch.nn.utils import clip_grad_norm_
from potts_decoder_nce import PottsDecoder
from torch.utils.data import DataLoader, RandomSampler
from functools import partial
import biotite.structure
from biotite.structure.io import pdbx, pdb
from biotite.structure.residues import get_residues
from biotite.structure import filter_backbone
from biotite.structure import get_chains
from biotite.sequence import ProteinSequence
from typing import Sequence, Tuple, List
import scipy
from tqdm import tqdm
import pandas as pd
import csv
import time
from torch.utils.tensorboard import SummaryWriter

#import pytorch_warmup as warmup
from torch.optim.lr_scheduler import ReduceLROnPlateau, MultiStepLR

##TURIN HPC
#sys.path.insert(1, "/Data/silva/esm/")

## EUROPA
#sys.path.insert(1, "/home/lucasilva/esm/")

##Lucas computer
sys.path.insert(1, "/home/luchinoprince/Dropbox/Old_OneDrive/Phd/Second_year/research/Feinauer/esm/")
import esm
#from esm.inverse_folding import util
import esm.pretrained as pretrained
from ioutils import read_fasta, read_encodings
from torch.nn.utils.rnn import pad_sequence
from collections import defaultdict
from Bio import SeqIO


In [3]:
B=3
N=6
q=21
M=20

encodings = torch.randn((B, N, 512))
#encodings = 0.1 * torch.ones((B, N, 512))
msas = torch.randint(low=0, high=21, size=(B,M,N))

embeddings = get_embedding(q)
msas_embedded = embeddings(msas)
#padding_mask = torch.zeros(size=(B, N),dtype=bool)
padding_mask = torch.randint(low=0, high=2, size=(B,N), dtype=bool)
padding_mask_inv = (~padding_mask)

input_encoding_dim = 512
q=21
param_embed_dim = 512
n_param_heads = 4
d_model = 128
n_heads = 2
n_layers = 2
device = 0
eta = 1e-3
dropout = 0.0
K = n_param_heads
device=1

decoder = PottsDecoder(q, n_layers, d_model, input_encoding_dim, param_embed_dim, n_heads, n_param_heads, dropout=dropout)

#model_path = '/media/luchinoprince/b1715ef3-045d-4bdf-b216-c211472fb5a2/Data/InverseFolding/Intermediate_Models/model_04_01_2023.pt'
#checkpoint = torch.load(model_path)

q=21
#args = checkpoint['args']
decoder.eval();


512
128


In [6]:
param_embeddings, fields, logZ = decoder.forward(encodings, padding_mask)


In [60]:
param_embeddings.shape

torch.Size([3, 4, 6, 21])

In [8]:
from nce import nce, loglik_potts, loglik_indep, sample_ind

In [9]:
fi = torch.rand(B, N, q)
fi = fi / torch.sum(fi,axis=2).unsqueeze(2)

In [10]:
samples_ind = sample_ind(fi, n_samples=M)

In [11]:
samples_ind.shape

torch.Size([20, 3, 6])

In [12]:
samples_ind = torch.transpose(samples_ind, 0, 1)
samples_ind_embedded = embeddings(samples_ind)

In [13]:
msas_embedded.shape

torch.Size([3, 20, 6, 21])

In [14]:
samples_ind_embedded.shape

torch.Size([3, 20, 6, 21])

In [15]:
samples_full = torch.concat([msas_embedded, samples_ind_embedded], axis=1)    


In [16]:
samples_full.shape

torch.Size([3, 40, 6, 21])

In [17]:
#V = torch.randn((B, K, N, q))
#fields = torch.randn((B, N, q))

In [61]:
#logZ = torch.log(torch.rand((B, )))

In [18]:
padding_mask.shape

torch.Size([3, 6])

In [20]:
#torch.min(loglik_indep(fi, samples_full, padding_mask))

lprobs: tensor([[[-2.5328, -2.6902, -3.2032, -3.6385, -2.8799, -2.5625],
         [-2.9088, -2.4310, -5.4418, -2.4707, -4.4348, -2.4016],
         [-3.2980, -5.5168, -2.3000, -2.5975, -2.5798, -2.7755],
         [-2.3477, -5.5168, -2.7695, -2.3389, -5.0443, -2.4016],
         [-2.4371, -2.3699, -4.8424, -3.1169, -3.1859, -2.3699],
         [-2.4886, -4.0973, -2.4428, -6.8887, -2.8243, -3.2987],
         [-3.9937, -2.7036, -3.4697, -2.2696, -2.9139, -5.0211],
         [-3.2206, -2.6207, -5.0968, -2.5040, -3.2096, -4.5869],
         [-3.3876, -2.4190, -5.0410, -4.5019, -2.9139, -2.4016],
         [-4.6455, -2.4310, -2.5890, -2.4707, -2.9139, -5.0211],
         [-2.4886, -2.7036, -3.5562, -2.5975, -2.6447, -3.2030],
         [-6.0371, -3.2282, -2.5890, -2.4707, -2.6447, -2.7111],
         [-4.6455, -2.4310, -5.0968, -2.9352, -4.0407, -3.2987],
         [-2.3477, -4.0973, -3.5562, -2.3389, -2.6447, -3.5834],
         [-2.7536, -4.3216, -4.8424, -2.8885, -3.3517, -4.5869],
         [-2.7260

tensor(-16.4685)

In [63]:
samples_full.shape

torch.Size([3, 40, 6, 21])

In [64]:
fields.shape

torch.Size([3, 6, 21])

In [65]:
padding_mask.shape

torch.Size([3, 6])

In [66]:
padding_mask_inv = (~padding_mask)
## (B, n_param_heads, N, q) * (B, 1, N, 1) = (B, K, N, q)
# set embeddings to zero where padding is present
param_embeddings2 = V * padding_mask_inv.unsqueeze(1).unsqueeze(3) 

fields2 = fields * padding_mask_inv.unsqueeze(-1)

In [67]:
F=fields2
V=param_embeddings2
K = V.shape[1]
B, M, _, _ = msas_embedded.shape

## (B, 1, K, M, q) * (B, M, 1, N, q) = (B, M, K, N, q)  ---> (B, M, K, N)
V_data = torch.sum(torch.unsqueeze(V, dim=1) * 
                    torch.unsqueeze(msas_embedded, dim=2), axis=-1)

## (B, M, K, N)  ---> (B, M, K)
S_k = torch.sum(V_data, axis=-1)
H_k = torch.sum(V_data**2, axis=-1)

## (B, M, K)  ---> (B, M)
E = torch.sum(S_k**2 - H_k, axis=-1)

## (B, 1, N, q) * (B, M, N, q) = (B, M, N, q) ---> (B, M, N)
Fi_data = torch.sum(torch.unsqueeze(F, dim=1) * msas_embedded, axis=-1)

## Fs is the field component of the sequence s: (B, M, N) --> (B, M) 
Fs = torch.sum(Fi_data, axis=-1)

In [69]:
fi.shape

torch.Size([3, 6, 21])

In [70]:
B, M, N

(3, 20, 6)

In [71]:
samples_full.shape

torch.Size([3, 40, 6, 21])

In [72]:
lprobs_i = torch.log(torch.sum(torch.unsqueeze(fi, dim=1) * samples_full, axis=-1))

In [74]:
lprobs_i.shape

torch.Size([3, 40, 6])

In [75]:
lprobs_i = lprobs_i * padding_mask_inv.unsqueeze(1)

In [76]:
lprobs_i

tensor([[[-2.7959, -2.8887, -0.0000, -3.2665, -3.2788, -0.0000],
         [-2.7959, -2.4124, -0.0000, -2.7914, -2.9853, -0.0000],
         [-4.1576, -2.9103, -0.0000, -3.3538, -2.7442, -0.0000],
         [-3.1487, -2.4124, -0.0000, -2.5161, -3.3445, -0.0000],
         [-2.7556, -5.0797, -0.0000, -4.6907, -2.7565, -0.0000],
         [-2.7959, -2.3490, -0.0000, -3.1399, -3.2788, -0.0000],
         [-3.5369, -2.3025, -0.0000, -4.6907, -3.8780, -0.0000],
         [-3.1141, -2.6681, -0.0000, -4.5578, -3.0445, -0.0000],
         [-2.4845, -2.3490, -0.0000, -2.6056, -3.2024, -0.0000],
         [-3.1164, -4.5041, -0.0000, -2.5763, -3.3445, -0.0000],
         [-2.3043, -4.4000, -0.0000, -2.5675, -2.8260, -0.0000],
         [-2.3352, -3.8065, -0.0000, -3.1399, -2.8260, -0.0000],
         [-2.3043, -2.5659, -0.0000, -4.4705, -2.7431, -0.0000],
         [-2.3622, -2.5433, -0.0000, -4.6907, -2.7106, -0.0000],
         [-2.3622, -2.6681, -0.0000, -2.4051, -3.9773, -0.0000],
         [-3.1141, -2.913

In [46]:
fi_mask = fi * padding_mask.unsqueeze(-1)

In [50]:
probs_i = torch.sum(torch.unsqueeze(fi_mask, dim=1) * samples_full, axis=-1)

In [51]:
probs_i

tensor([[[0.0569, 0.0050, 0.0000, 0.0299, 0.0982, 0.0000],
         [0.0569, 0.0645, 0.0000, 0.0503, 0.0048, 0.0000],
         [0.0531, 0.0403, 0.0000, 0.0983, 0.0003, 0.0000],
         [0.0109, 0.0645, 0.0000, 0.0180, 0.0225, 0.0000],
         [0.0815, 0.0557, 0.0000, 0.0749, 0.0655, 0.0000],
         [0.0569, 0.0844, 0.0000, 0.0339, 0.0982, 0.0000],
         [0.0218, 0.0212, 0.0000, 0.0749, 0.0504, 0.0000],
         [0.0459, 0.0189, 0.0000, 0.0408, 0.0469, 0.0000],
         [0.0037, 0.0844, 0.0000, 0.0248, 0.0068, 0.0000],
         [0.0811, 0.0049, 0.0000, 0.0045, 0.0225, 0.0000],
         [0.0594, 0.0013, 0.0000, 0.0264, 0.0965, 0.0000],
         [0.0893, 0.0512, 0.0000, 0.0339, 0.0965, 0.0000],
         [0.0594, 0.0264, 0.0000, 0.0128, 0.0258, 0.0000],
         [0.0002, 0.0184, 0.0000, 0.0749, 0.0836, 0.0000],
         [0.0002, 0.0189, 0.0000, 0.0679, 0.0435, 0.0000],
         [0.0459, 0.0471, 0.0000, 0.0180, 0.0504, 0.0000],
         [0.0510, 0.0050, 0.0000, 0.0408, 0.0982, 0.0000

In [44]:
probs_i.shape

torch.Size([3, 40, 6])

In [40]:
Fs

tensor([[ 2.4179,  1.0070, -0.4219, -3.9579, -2.7612, -0.4937, -4.3845,  0.2234,
         -0.3561, -0.3228, -1.6439, -0.3902,  0.1866,  0.0304,  1.9634, -0.2119,
          0.3968, -4.5075, -0.9949,  5.5444],
        [-1.4001,  2.0268, -1.0783,  0.0530, -0.6134, -0.2297,  0.0709,  1.3258,
          0.3986,  1.1676, -1.9901,  0.2156, -0.5675, -0.5437, -0.2863, -1.9482,
         -2.4567, -1.1607, -1.2261,  0.8671],
        [-0.2373, -0.8207, -0.6469,  1.3175, -0.9699, -0.4068,  0.8829,  1.2168,
         -1.1801, -0.9565, -1.4848, -1.1996, -0.6022,  1.9340, -0.3521, -1.2212,
         -2.0115, -2.1553, -1.1180,  0.2196]])

In [23]:
log_lik, ham_s = loglik_potts(param_embeddings2, fields2, logZ, samples_full) 

In [24]:
ham_s

tensor([[ -0.2376,  -3.7111,  -4.4863,  -2.7635,   0.8379,  -0.0753,  -0.6168,
           3.4276,   9.2806,  -2.9406,   1.2479,  -1.1388,  -4.7617,  -2.3241,
           0.3153,  -0.3997,   7.0578, -11.8621,   4.1278,   0.3478,  -3.8795,
           0.6578,  -0.1433,   2.4244,  -0.4940,  -3.0831,  -7.9304,  -2.1165,
          -5.8685,  -1.9086,   0.1472,   0.8621,   1.9392,   2.3627,  -5.6857,
          -8.7938,   0.1421,   9.6357,  -2.3659,   3.7855],
        [ -5.2727,  -2.4764,   0.6880,  -1.0528,  11.7529,  -1.4121,  -2.1735,
           6.6860,   0.1185,   0.8783,  -3.3314,   4.2387,  -0.7928,  -0.7989,
          -3.4933,  -2.2688,  -2.4604,  -7.2371,   3.3306,  -0.3710,  -0.1299,
           2.5268,  -0.9580,   0.7442,   0.4685,  -0.0887, -10.8362, -11.0541,
          -6.4370, -11.0569,  -4.1219,  -1.6273,  -2.3082,  -6.5055,   0.9592,
           7.9138,  -4.4997,  -7.5472,   5.1019,  -1.5960],
        [ -1.7896,  -2.7972,   2.0103,  -0.4200,  -6.4310,  -0.2355,  -0.1247,
           

In [86]:
logZ

tensor([-0.6494, -0.9912, -0.1080])

In [18]:
G = loglik_potts(V, fields, logZ, samples_full) - loglik_indep(fi, samples_full)

In [17]:
mask_noise = torch.zeros((B,2*M))
mask_noise[:, 0:M] = 1

In [18]:
nce(V, fields, Z, msas_embedded, samples_ind_embedded, mask_noise, fi, N, M, q)

tensor([[0.0000e+00, 8.3446e-07, 1.8893e+01, 1.8696e+01],
        [0.0000e+00, 0.0000e+00, 1.7686e+01, 1.7921e+01],
        [0.0000e+00, 0.0000e+00, 2.1240e+01, 2.3362e+01]])